<div style="background-color:Brown;"><h1 style="color:Orange;text-align:center; font-size:40px;">  Web Scraping - Mission to Mars</h1> </div>

### Importing dependencies

In [1]:
import sys
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import requests
from webdriver_manager.chrome import ChromeDriverManager
import urllib.request
from selenium import webdriver  # using selenium to obtain the whole webpage content due to JS implementation in the website
!{sys.executable} -m pip install chromedriver_autoinstaller
import chromedriver_autoinstaller # needed to the proper operation of Selenium

In [2]:
chromedriver_autoinstaller.install() # Automatically finds the correct Driver based on the currently installed chromes version 
executable_path = {'executable_path': ChromeDriverManager().install()} # Executable path needed for splinter's correct operation

[WDM] - ====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
INFO:WDM:Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
INFO:WDM:Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\jich-\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache
INFO:WDM:Driver [C:\Users\jich-\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


<h1 style="color:blue;text-align:center; font-size:40px;"> Scraping </h1>

<h2 style="color:green;text-align:center;"> NASA Mars News </h2>

In [3]:
# Mars News site
url = "https://redplanetscience.com/"

In [4]:
# instantiating the webdriver for Chrome!!!
opt = webdriver.ChromeOptions()
opt.add_argument("--start-minimized")
driver = webdriver.Chrome(options=opt)
# Getting the webpage content
driver.get(url)
# parsing our html plain text to a BS object
soup = BeautifulSoup(driver.page_source, 'html.parser')
# Closing the driver(chrome instance)
driver.quit()
print(soup.prettify())

<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/css/bootstrap.min.css" integrity="sha384-giJF6kkoqNQ00vy+HMDP7azOuL0xtbfIcaT9wjKHr8RbDVddVHyTfAAsrekwKmP1" rel="stylesheet"/>
  <link href="css/font.css" rel="stylesheet" type="text/css"/>
  <link href="css/app.css" rel="stylesheet" type="text/css"/>
  <link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.10.0/css/all.css" integrity="sha384-AYmEC3Yw5cVb3ZcuHtOA93w35dYTsvhLPVnYs9eStHfGJvOvKxVfELGroGkvsg+p" rel="stylesheet"/>
  <title>
   News - Mars Exploration Program
  </title>
 </head>
 <body>
  <div class="col-md-12">
   <div class="row">
    <nav class="navbar navbar-expand-lg navbar-light fixed-top">
     <div class="container-fluid">
      <a class="navbar-brand" href="#">
       <img src="image/nasa.png" width="80"/>
       <span class="logo">
        MA

In [5]:
# Retrieve the parent divs for all articles
results = soup.find_all('div', class_='list_text')[0]  # using index 0 to get only the lastest news Title
# Storing the news title
news_title = results.find('div',class_='content_title').text
news_p = results.find('div',class_='article_teaser_body').text

In [6]:
print(f"Title: {news_title}")
print(f"Paragraph: {news_p}")

Title: Alabama High School Student Names NASA's Mars Helicopter
Paragraph: Vaneeza Rupani's essay was chosen as the name for the small spacecraft, which will mark NASA's first attempt at powered flight on another planet.


<h2 style="color:green;text-align:center;"> JPL Mars Space Images - Featured Image </h2>

In [7]:
# browser = Browser('chrome', **executable_path, headless=False)